# Multiple Sequence Alignment of 16S Ribosomal RNA Genes in Bacteria

## Introduction to Multiple Sequence Alignment (MSA)

Multiple sequence alignment (MSA) is a method used to align three or more biological sequences (protein, DNA, or RNA) to identify regions of similarity that may indicate functional, structural, or evolutionary relationships. MSA is crucial for:
- Identifying conserved sequences across different species
- Inferring phylogenetic relationships
- Predicting secondary and tertiary structures of proteins and RNAs
- Understanding evolutionary changes and functional divergence

In this project, we focus on the 16S ribosomal RNA (rRNA) genes from various bacteria. The 16S rRNA gene is a highly conserved component of the small subunit of prokaryotic ribosomes and is commonly used in phylogenetic studies. By leveraging MSA, we aim to explore the conserved regions and variations among these bacterial sequences. The steps will include sequence retrieval, performing MSA using different tools, and analyzing the results comprehensively.


**Sequence Retrieval:**                                                                                                                                       
We'll retrieve the 16S rRNA gene sequences for various bacteria using NCBI Entrez.

In [1]:
from Bio import Entrez, SeqIO

# Setting up email address to NCBI
Entrez.email = "karkidholipankaj@gmail.com"

# Searching for 16S rRNA sequences from bacteria
search_term = "16S ribosomal RNA[Gene] AND Bacteria[Organism]"
handle = Entrez.esearch(db="nucleotide", term=search_term, retmax=20)
record = Entrez.read(handle)
handle.close()

# Printing the number of sequences found
print("Number of sequences found:", record["Count"])

# Fetching the sequences
id_list = record["IdList"]
print("Number of sequence IDs retrieved:", len(id_list))

handle = Entrez.efetch(db="nucleotide", id=id_list, rettype="fasta", retmode="text")
sequences = list(SeqIO.parse(handle, "fasta"))
handle.close()

# Filtering sequences to be within the expected length for 16S rRNA
filtered_sequences = []
for seq in sequences:
    if 1200 <= len(seq.seq) <= 1600:
        filtered_sequences.append(seq)

# Checking the number of sequences retrieved and filtered
print("Number of sequences fetched:", len(sequences))
print("Number of sequences after filtering:", len(filtered_sequences))

# Saving filtered sequences to a file
output_handle = open("bacterial_16s_rrna_filtered.fasta", "w")
SeqIO.write(filtered_sequences, output_handle, "fasta")
output_handle.close()

# Calculating the length of each filtered sequence
for record in filtered_sequences:
    print("Length of " + record.id + ": " + str(len(record.seq)))

Number of sequences found: 141
Number of sequence IDs retrieved: 20
Number of sequences fetched: 16
Number of sequences after filtering: 10
Length of FN185731.1: 1513
Length of FJ184385.1: 1384
Length of DQ103668.1: 1488
Length of DQ103667.1: 1466
Length of DQ103666.1: 1477
Length of DQ103665.1: 1464
Length of DQ103664.1: 1463
Length of DQ103663.1: 1481
Length of DQ103662.1: 1458
Length of DQ103661.1: 1415


**Performing Multiple Sequence Alignment (MSA)**                                                                                       
We will now perform multiple sequence alignment (MSA) using ClustalW. This process aligns the sequences to identify regions of similarity, which may indicate functional, structural, or evolutionary relationships between the sequences.

In [5]:
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO

# Path to ClustalW executable
clustalw_exe = r"C:\\Program Files (x86)\\ClustalW2\\clustalw2.exe"

# Defining input and output files
in_file = "bacterial_16s_rrna_filtered.fasta"
out_file = "bacterial_16s_rrna_filtered.aln"

# Setting up the ClustalW command line
clustalw_cline = ClustalwCommandline(clustalw_exe, infile=in_file)

# Running ClustalW
stdout, stderr = clustalw_cline()

# Loading the alignment file
alignment = AlignIO.read("bacterial_16s_rrna_filtered.aln", "clustal")

# Printing the alignment after truncating
for record in alignment:
    truncated_seq = str(record.seq)[:90]
    if len(record.seq) > 90:
        truncated_seq += "....."
    print(f">{record.id} {truncated_seq}")
    

>DQ103664.1 -------------------ATTGAACGCTGGCGGCATGCCTAACACATGCAAGTCGAACGGT--AACGCGGGAGCTTGCTCCT-------.....
>DQ103663.1 -------------------ATTGAACGCTGGCGGCATGCTTAACACATGCAAGTCGTGGGGC--AGCAGGTCAGACAATCCCTTCGGGGT.....
>DQ103665.1 -------------------ATTGAACGCTAGCGGCATGCCTGACACATGCAAGTCGAACGGC--AGCGCGGGAAAGCTTGCTTTCC----.....
>DQ103662.1 -------------------AATAAACGTTGGCGGCGTGCCTAACACATGCAAGTCG--------AGCGAGAAAGCTTCTTCGGGAG----.....
>DQ103667.1 -------------------AATGAACGCTGGCGGCGTGCCTAACACATGCAAGTCGAGCGAG--AAAGCCC----CCTTCGGGGGG----.....
>DQ103666.1 -------------------AATGAACGCTGGCGGCGTGCCTAACACATGCAAGTCGAGCGAG--AAAGTCA----CCTTCGGGTGG----.....
>DQ103668.1 -------------------AATGAACGCTGGCGGCGTGCTTAACACATGCAAGTCGAACGAG--AAAGCCGAAAGCTTGCTTTTGG----.....
>FJ184385.1 -----------------------------------------------TGCAAGTCG--------AACGGAA----TCCTTAG--------.....
>FN185731.1 AGTTTTGATTATGGCTCAGGACGAACGCTGGCGGCATGCCTAATACATGCAAGTCGAACGCGTCTTCGTTAACTGAAGTGCTTGCACGGA.....
>DQ103661.1 ----------------

**Consensus Sequence:**

In [4]:
from Bio.Align import AlignInfo

# Generating summary of the alignment
summary_align = AlignInfo.SummaryInfo(alignment)

# Getting the consensus sequence
consensus = summary_align.dumb_consensus()

# Printing the consensus sequence
print(consensus)

AGTTTTGATTATGGCTCAGAXTGAACGCTGGCGGCXTGCCTAACACATGCAAGTCGAXCGXXTCAXCGXXXXAXXXXXTXXXXXXGXGGXXXXTXAXXXXXXGXXXAGXGGCGXACGGGTGAGTAACXCGTXGGTAAXCTXCCXXXXXGXXXGGGATAACXCXXXGAAAXXXGXGCTAATACCGXATXXXXXCCXGXXXXCTXXGGCXXXXXGGXXXAAAGXXGGXXTCTXCATXXXXGCTXXXGCXXXXGGATGXGCCXGCGTCCXATTAGCTXGTTGGTGXGGTAAXGGCTCACCAAGGCXACGATXGGTAGCTGGTCTGAGAGGATGATCAGCCACACTGGGACTGAGACACGGCCCAGACTCCTACGGGAGGCAGCAGTGGGGAATXTTGXXCAATGGGCGXAAGCCTGAXXCAGCAAXGCCGCGTGXGXGAAGAAGGCXXTCGGGTXGTAAAXCXCTXTXXXXXGGGAAGAAXXXXXXXXXGXTTAAXAXXXXXXXXXXXTGACGGTACCXXXXGAXXAAGCACCGGCTAACTCCGTGCCAGCAGCCGCGGTAATACGGAGGGTGCAAGCGTTATTCGGAATXACTGGGCGTAAAGXGXXCGXAGGCGGXXXXXTAAGTCXGXTGTGAAAGCCCXXGGCTXAACCXXGGAAXXGCAXTXGAXACTGXXXXXCTXGAGTXXGGXAGAGGXXAGXGGAATTCCXGGTGTAGCGGTGAAATGCGTAGATATCXGGAXGAACACCXGTGGCGAAGGCGGCTXXCTGGXCCXXXACTGACGCTGAGGXXCGAAAGCGTGGGXAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGXXXXCTAGXTGTTTGGGXXXXXXXTXXXXXXXXTXAGTGXCGXAGCTAACGCGTTAAGXXXXCCGCCTGGGGAGTACGGXCGCAAGXXTXAAACTCAAAGGAATTGACGGGGGCCCGCACAAGCGGTGGAGCATGTGGTTTAATTCGATGCAA

The consensus sequence generated from aligning 10 sequences of 16S rRNA shows both conserved and variable regions. Conserved regions, represented by consistent nucleotide patterns, are likely functionally significant for the 16S rRNA's role in ribosomal RNA structure and function. These regions are crucial for maintaining the integrity and function of the ribosome. In contrast, segments with frequent ambiguous bases (X) indicate variability, reflecting genetic diversity among the sequences or regions with less evolutionary pressure to conserve specific nucleotides. This variability can provide insights into the evolutionary relationships and adaptations among the organisms from which these sequences were derived.

**References:**    
                                                                                                                              
Janda, J. M., & Abbott, S. L. (2007). “16S rRNA Gene Sequencing for Bacterial Identification in the Diagnostic Laboratory: Pluses, Perils, and Pitfalls.” Journal of Clinical Microbiology, 45(9), 2761–2764. doi:10.1128/JCM.01228–07.                                                                                                                                        

Thompson, J. D., Higgins, D. G., & Gibson, T. J. (1994). “CLUSTAL W: improving the sensitivity of progressive multiple sequence alignment through sequence weighting, position-specific gap penalties and weight matrix choice.” Nucleic Acids Research, 22(22), 4673–4680. doi:10.1093/nar/22.22.4673.